In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Part II: Model Building

Here you try your hand at model building to predict appointment no shows.

### Preprocessing

Package 'proj2_lib' now includes code to carry out preprocessing steps from part I. Here's how to use it:

In [16]:
import proj2_lib.util as utils

First, it includes a dictionary used for configuring path and file names
used through the project

In [17]:
utils.file_config

{'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl',
 'objstore_path': 'objects',
 'processed_data_path': 'processed_data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'raw_data_path': 'data',
 'test_csv': 'test_set.csv',
 'train_csv': 'train_set.csv'}

`feature_pipeline_file`: file storing the preprocessing pipeline used for preparing the feature matrix

`labels_pipeline_file`: file storing the preprocessing pipeline used for
preparing labels

`objstore_path`: directory to store python objects to disk

`processed_data_path`: directory containing processed data

`raw_data_csv`: name of the csv download from Kaggle

`raw_data_path`: directory containing raw data

`test_csv`: name of csv file containing test set

`train_csv`: name of csv file containing train set

You can change these paths and names to suit your project directory structure if you need so. E.g.,

In [18]:
file_config = utils.file_config
#config['raw_data_path'] = "some_other_directory"

First step is to create train test sets. Code is in file `proj2_lib/util.py` function `make_train_test_sets`. You
can edit that function as needed to include your own part I code if you so desire. The result will be to 
create files `train_set.csv` and `test_set.csv` in your `processed_data` directory (unless you change any of the entries in the configuration directory as above).

In [19]:
# ONLY NEED TO RUN THIS STEP ONCE (switch this to True to run it)
RUN_MAKE_TRAIN_TEST_FILES = False
if RUN_MAKE_TRAIN_TEST_FILES:
    utils.make_train_test_sets(config=file_config)

Next step is to fit the preprocessing pipelines. This is done in file `proj2_lib/preprocess.py`. Again you can edit code as needed in that file to incorporate your part I solution as you wish. The result will be to create files `feature_pipeline.pkl` and `labels_pipeline.pkl` containing the fit preprocessing pipelines we can then use to preprocess data.

In [20]:
import proj2_lib.preprocess as preprocess

# ONLY NEED TO RUN THIS STEP ONCE
RUN_FIT_PREPROCESSING = True
if RUN_FIT_PREPROCESSING:
    preprocess.fit_save_pipelines(config=file_config)

Finally, once we do that, we can get a training matrix and labels:

In [21]:
train_X, train_y = preprocess.load_train_data(config=file_config)

In [22]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


In [23]:
train_X

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Model Building

Using `sklearn` fit:
    - DecisionTree classifier
    - RandomForest classifier
    - Linear SVM classifier
    - SVM with Radial Basis Kernel classifier
    
Use default parameters for now.
Using 10-fold cross validation report both accuracy and AUC for each of the above four models.

QUESTION: Should you use accuracy or AUC for this task as a performance metric?

_ANSWER HERE_

In [32]:
# build your models here
import numpy as np 
from sklearn.tree import DecisionTreeClassifier



tree_clf = DecisionTreeClassifier()
tree_clf_fitted = tree_clf.fit(train_X, train_y)

#tree_predictions = tree_reg.predict(train_X)
#accu = accuracy_score(train_y, tree_predictions.astype(int))
#print(accu)



In [42]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier()
forest_clf_fitted = forest_clf_fit=forest_clf.fit(train_X,train_y)


In [43]:
from sklearn.svm import LinearSVC
SVM_clf = LinearSVC()
SVM_clf_fitted = SVM_clf_fit=SVM_clf.fit(train_X,train_y)


In [48]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

def measure_performance(X,y,clf, show_accuracy=True, show_AUC=True, show_classification_report=True, show_confusion_matrix=True):

    accuracy_scores = cross_val_score(clf, X, y, 
                        scoring="accuracy", cv=10)
    
    
    AUC_scores = cross_val_score(clf, X, y, 
                        scoring="roc_auc", cv=10)
    
    y_pred=clf.predict(X)   
    
    print ("Fitted model:")
    print (clf,"\n")
    if show_accuracy:
        print ("Accuracy:")
        print (accuracy_scores,"\n")
        print ("Mean Accuracy")
        print (accuracy_scores.mean(), "\n")
        
    if show_AUC:
        print ("AUC:")
        print (AUC_scores,"\n")        
        print ("Mean AUC")
        print (AUC_scores.mean(), "\n")        
        
    if show_classification_report:
        print ("Classification report")
        print (metrics.classification_report(y,y_pred),"\n")
        
    if show_confusion_matrix:
        print ("Confusion matrix")
        print (metrics.confusion_matrix(y,y_pred),"\n")
        

In [49]:
# decision tree 
measure_performance(train_X,train_y,tree_clf_fitted, show_classification_report=True, show_confusion_matrix=True)

Fitted model:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best') 

Accuracy:
[ 0.74351044  0.73853971  0.74130123  0.74008616  0.73445267  0.73798741
  0.74060981  0.73298719  0.74171454  0.73442333] 

Mean Accuracy
0.738561247486 

AUC:
[ 0.58489188  0.58400847  0.58266902  0.57965731  0.58077423  0.57645151
  0.58484312  0.5759704   0.59008413  0.57904496] 

Mean AUC
0.581839502662 

Classification report
             precision    recall  f1-score   support

         -1       0.98      1.00      0.99     72246
          1       0.99      0.91      0.95     18280

avg / total       0.98      0.98      0.98     90526
 

Confusion matrix
[[72107   139]
 [ 1622 16658]] 



In [50]:
# decision tree 
measure_performance(train_X,train_y,forest_clf_fitted, show_classification_report=True, show_confusion_matrix=True)

Fitted model:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False) 

Accuracy:
[ 0.78106705  0.77742185  0.77388711  0.77985198  0.77245112  0.77830553
  0.77662395  0.77750773  0.77596111  0.77087936] 

Mean Accuracy
0.776395680682 

AUC:
[ 0.69874486  0.69129091  0.68977096  0.70241064  0.69211894  0.69545933
  0.69484385  0.68011177  0.69922749  0.68882718] 

Mean AUC
0.693280593178 

Classification report
             precision    recall  f1-score   support

         -1       0.96      0.99      0.98     72246
          1       0.96      0.84      0.90     18280

avg / total       0.96      0.96      0.96     90526
 

Confusion matrix
[[71608   638]
 [ 2880 

In [51]:
# SVM
measure_performance(train_X,train_y,SVM_clf_fitted, show_classification_report=True, show_confusion_matrix=True)

Fitted model:
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0) 

Accuracy:
[ 0.79608969  0.79509555  0.79675246  0.79608969  0.79586877  0.796642
  0.79595669  0.79750331  0.79639859  0.79617764] 

Mean Accuracy
0.796257439923 

AUC:
[ 0.6653065   0.66874815  0.67107198  0.66971429  0.66174169  0.67429255
  0.66172163  0.66429367  0.67142765  0.66481516] 

Mean AUC
0.667313327537 

Classification report
             precision    recall  f1-score   support

         -1       0.80      1.00      0.89     72246
          1       0.34      0.01      0.02     18280

avg / total       0.71      0.80      0.71     90526
 

Confusion matrix
[[71911   335]
 [18111   169]] 



###  Should you use accuracy or AUC as a metric for this task?

I think we should AUC. Because the data set is strongly biased towards the showing up. If data set is biased we can get good accuracy just using a dummy model which says all patients will show up.   



## Model Tuning

Based on the above, choose two methods and fit a tuned model:
- use 5-fold cross validation for model selection
- use 10-fold cross validation for model assessment (based on appropriate performance metric)


Report estimated performance for both tuned classifiers


In [54]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [5, 25, 50, 75, 100], 'max_features': [5, 10, 15, 20]},
    {'bootstrap': [False], 'n_estimators': [5, 25, 50, 75, 100], 'max_features': [5, 10,15, 20]}
]


forest_clf = RandomForestClassifier()

forest_grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                          scoring="roc_auc")
forest_grid_search.fit(train_X, train_y)

print ("Random Forest Best params: ", forest_grid_search.best_params_)
print("Random Forest Best estimator score {}".format(forest_grid_search.best_score_))

Random Forest Best params:  {'max_features': 15, 'n_estimators': 100}
Random Forest Best estimator 0.7201803984727179


In [56]:
#from sklearn import svm, grid_search

#def svc_param_selection(X, y, nfolds):
#    Cs = [0.001, 0.01, 0.1, 1, 10]
#    gammas = [0.001, 0.01, 0.1, 1]
#    param_grid = {'C': Cs, 'gamma' : gammas}
#    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
#    grid_search.fit(X, y)
#    grid_search.best_params_
#    return grid_search.best_params_

Cs = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': Cs}

SVM_clf = LinearSVC()
SVM_grid_search = GridSearchCV(SVM_clf, param_grid, cv=5,
                          scoring="roc_auc")
SVM_grid_search.fit(train_X,train_y)
print (" Linear SVM Best params: ", SVM_grid_search.best_params_)
print("Linear SVM Best estimator score {}".format(SVM_grid_search.best_score_))


 Linear SVM Best params:  {'C': 0.001}
Linear SVM Best estimator score 0.66929850565305


### Linear SVM with Gradient Descent

In [65]:
import numpy as np

# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    w = np.full((nfeatures), np.random.randn())
    b = np.full((1), np.random.randn())
    return w, b

# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    loss = np.sum(pos_part(1.0 - y*f))    
    penalty = lam * np.dot(w,w)
    return loss + penalty

# compute the signed distances
# based on current model estimates
# w and b
# YOU NEED TO IMPLEMENT THIS
def _get_signed_distances(X, w, b):
    nobs = X.shape[0]
    f = np.full(nobs, 0.0)
    f = np.multiply(X,w) + b
    return f

# compute gradients with respect to w and b
# YOU NEEED TO IMPLEMENT THIS
gradient_part1 = np.vectorize(lambda u: 0. if u >= 1. else -1.)
def _get_gradients(f, X, y, w, b, lam):
    #nfeatures = X.shape[1]
    #gw = np.full((nfeatures), 0.)
    #gb = 0.
    
    yf = y * f
    t = gradient_part1(yf)
    ty = t * y 
    gb = np.sum(ty)
    
    tmp = np.multiply(X.T, ty).T
    gw = 2*lam*w + np.sum(tmp , axis = 0)
    
    return gw, gb

# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# n_iter: numer of iterations
# eta: learning rate
def fit_svm(X, y, lam, n_iter=100, eta=.4):
    nexamples, nfeatures = X.shape
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % 10 == 0:
            obj = _get_objective(f, y, w, lam)
            eta = eta / 2.0
            print("it: %d, obj %.2f" % (k, obj))
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        w = w - eta * gw
        b = b - eta * b
    return w, b

In [68]:
w,b = fit_svm(train_X, train_y, 1.0, n_iter=100)

ValueError: operands could not be broadcast together with shapes (90526,) (90526,101) 

In [63]:
import numpy as np

# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    w = np.full((nfeatures), 0.0)
    b = np.full((1), 0.0)
    return w, b

# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    loss = np.sum(pos_part(1.0 - y*f))
    penalty = lam * np.dot(w,w)
    return loss + penalty

# compute the signed distances
# based on current model estimates
# w and b
# YOU NEED TO IMPLEMENT THIS
def _get_signed_distances(X, w, b):
    nobs = X.shape[0]
    f = np.full(nobs, 0.0)
    return f

# compute gradients with respect to w and b
# YOU NEEED TO IMPLEMENT THIS
def _get_gradients(f, X, y, w, b, lam):
    nfeatures = X.shape[1]
    gw = np.full((nfeatures), 0.)
    gb = 0.
    return gw, gb

# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# n_iter: numer of iterations
# eta: learning rate
def fit_svm(X, y, lam, n_iter=100, eta=.4):
    nexamples, nfeatures = X.shape
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % 10 == 0:
            obj = _get_objective(f, y, w, lam)
            eta = eta / 2.0
            print("it: %d, obj %.2f" % (k, obj))
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        w = w - eta * gw
        b = b - eta * b
    return w, b

In [64]:
w,b = fit_svm(train_X, train_y, 1.0, n_iter=100)

it: 0, obj 90526.00
it: 10, obj 90526.00
it: 20, obj 90526.00
it: 30, obj 90526.00
it: 40, obj 90526.00
it: 50, obj 90526.00
it: 60, obj 90526.00
it: 70, obj 90526.00
it: 80, obj 90526.00
it: 90, obj 90526.00
